In [115]:
# Colab 相关设置项
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes


In [0]:
import collections
import random
import imp

In [118]:
%mkdir ch12
!touch ch12/__init__.py

mkdir: cannot create directory ‘ch12’: File exists


In [52]:
%%writefile ch12/binary_search_tree.py
"""二叉搜索树"""
import random  # 后续删除结点时，可以随机的选择前驱和后继


class BinaryTreeNode():
  """二叉搜索树结点类"""
  def __init__(self, parent=None, left=None, right=None, key=None):
    self.parent = parent
    self.left = left
    self.right = right
    self.key = key
  
  def __repr__(self):
    return 'None' if self.key == None else str(self.key)


class BinarySearchTree():
  """二叉搜索树类"""
  def __init__(self, root=None):
    self.root = root

  def __repr__(self):
    """按逆时针方向旋转 90 度打印二叉树"""
    def helper(x, i):
      res = ''
      if x != None:
        res += helper(x.right, i+1)
        res += "\n" + "|  " * i + "{}\n".format(x)
        res += helper(x.left, i+1)
      return res
    
    return 'None' if self.root == None else helper(self.root, 0)

Overwriting ch12/binary_search_tree.py


## 12.0 二叉树的相关定义

###### 二叉树的深度与高度

- [参考网址](https://stackoverflow.com/questions/2603692/what-is-the-difference-between-tree-depth-and-height)
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221213230.png width=400>

## 12.1 什么是二叉搜索树

###### 二叉搜索树

 - 以一颗二叉树来组伿
  - 每个结点除了 $key$ 和卫星数据外，还包含属性 $left$, $right$ 和 $p$
  - 二叉搜索中树中的关键字总是以满足**二叉搜索树**性质的方式来存储
    - 设 $x$ 是二叉搜索树中的一个结点。如果$y$是 $x$ 左子树中的一个结点，则 $y.key \le x.key$ ；如果 $y$ 是 $x$ 左子树中的一个结点，那么 $y.key \ge x.key$
  - 示意图 
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200211143416.png width=600>
  $$$$

##### 二叉树的遍历
  - 根据根节点被遍历的顺序可以分为中序遍历、前序遍历和后续遍历
  - 在三种遍历中，左子树中的节点总是先于右子树中的节点被遍历
  - 中序遍历可以按序输出二叉树中的所有关键字

###### 中序遍历的代码实现

In [53]:
%%writefile -a ch12/binary_search_tree.py


  def inorder_walker(self, x):
    while x != None:
      self.inorder_walker(x.left)
      print(x.key, end=" ")
      x = x.right  # 尾递归技术

Appending to ch12/binary_search_tree.py


In [0]:
import ch12.binary_search_tree
from ch12.binary_search_tree import BinaryTreeNode, BinarySearchTree
# 构建一颗二叉搜索树
nodes = []
for i in [15, 6, 18, 3, 7, 17, 20, 2, 4]:
  nodes.append(BinaryTreeNode(key=i))
for i, node in enumerate(nodes):
  parent = (i - 1) // 2
  left = 2 * i + 1
  right = 2 * i + 2
  if parent >= 0:
    node.parent = nodes[parent]
  if left < len(nodes):
    node.left = nodes[left]
  if right < len(nodes):
    node.right = nodes[right]

a = BinaryTreeNode(key=13)
b = BinaryTreeNode(key=9)

nodes[4].right = a
a.parent = nodes[4]
a.left = b
b.parent = a

bst = BinarySearchTree(nodes[0])

In [55]:
print("二叉搜索树为:\n {}".format(bst))
print("中序遍历的结果为：")
bst.inorder_walker(bst.root)

二叉搜索树为:
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2

中序遍历的结果为：
2 3 4 6 7 9 13 15 17 18 20 

###### 定理 12.1
- 遍历一颗拥有 $n$ 个节点的二叉树，需要 $\Theta(n)$ 的时间

### 练习

#### 12.1-3 
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200212163108.png width=700>

###### 使用栈作为辅助结构
- 栈用来存储暂时不会输出的结点，同时借助栈先进后出的特性，来保证遍历的顺序
- 步骤
  1. 先沿着左子节点向下遍历，同时将各个节点压入栈中，一直遍历到没有左子节点的节点 $x$, 然后输出 $x.key$
  2. 然后将 $x$ 置为 $x.right$，如果其存在，则重复 1 种的操作，否则出栈，并将值赋给 $x$
  3. 当栈不为空或者 $x$ 不为 $None$ 时，则继续循环

In [0]:
def inorder_walker_iter_stack(x):
  stack = collections.deque()
  while len(stack) or x != None:
    while x != None:
      stack.append(x)
      x = x.left
    x = stack.pop()
    print(x.key, end=' ')
    x = x.right


In [57]:
print("二叉搜索树为:\n {}".format(bst))
print("中序遍历的结果为：")
inorder_walker_iter_stack(bst.root)

二叉搜索树为:
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2

中序遍历的结果为：
2 3 4 6 7 9 13 15 17 18 20 

###### 不使用栈
- 不使用栈，需要用 $prev$ 来记录节点遍历的状态
- 步骤
  1. 将 $prev$ 置为 $None$，通过将 $x$ 置为 $x.left$ 来一直向其左子节点遍历，直至 $x.left$ 为 $None$，此时可输出 $x$ 的值
  2. - 如果 $x.right$ 存在，则将 $x$ 置为 $x.right$， $prev$ 置 $None$，并重复 1 中的操作
    - 如果 $x.right$ 不存在，则需要向其父辈节点遍历，为了判断 $x$ 是其父辈节点的左子树还是右子树，需要将 $x$ 的值存入 $prev$ 中，并
  将 $x$ 指向 $x.parent$，如果 $x.parent$ 不存在，则可以结束整个函数 
  3. 
    - 如果 $x$ 是从左子树遍历上来的，则打印当前节点，并继续 2 中的操作
    - 如果 $x$ 是从右子树遍历上来的，则继续向上一层遍历，即更新 $x$ 和 $prev$ 后重复 3 中的操作

In [0]:
def inorder_walker_iter(x):
  if x == None:
    print(None)
    return
  prev = None
  while True:

    if prev == None:
      while x.left != None:
        x = x.left
      print(x.key, end=" ")
      if x.right != None:
        x = x.right
        continue
    else:
      if x.left == prev:
        print(x.key, end=" ")
        if x.right != None:
          x = x.right
          prev = None
          continue
    
    if x.parent != None:
      prev = x
      x = x.parent
    else:
      return

In [59]:
print("二叉搜索树为:\n {}".format(bst))
print("中序遍历的结果为：")
inorder_walker_iter(bst.root)

二叉搜索树为:
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2

中序遍历的结果为：
2 3 4 6 7 9 13 15 17 18 20 

#### 12.1-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200212210300.png width=800>

###### 先序遍历

In [60]:
%%writefile -a ch12/binary_search_tree.py


  def preorder_walker(self, x):
    while x != None:
      print(x.key, end=" ")
      self.preorder_walker(x.left)
      x = x.right

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode


In [62]:
bst = BinarySearchTree(bst.root)
print("二叉搜索树为:\n {}".format(bst))
print("先序遍历的结果为：")
bst.preorder_walker(bst.root)

二叉搜索树为:
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2

先序遍历的结果为：
15 6 3 2 4 7 13 9 18 17 20 

###### 后序遍历

In [63]:
%%writefile -a ch12/binary_search_tree.py


  def postorder_walker(self, x):
    if x != None:
      self.postorder_walker(x.left)
      self.postorder_walker(x.right)
      print(x.key, end=" ")

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode

In [65]:
bst = BinarySearchTree(bst.root)
print("二叉搜索树为:\n {}".format(bst))
print("后序遍历的结果为：")
bst.postorder_walker(bst.root)

二叉搜索树为:
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2

后序遍历的结果为：
2 4 3 9 13 7 6 17 20 18 15 

## 12.2 查询二叉搜索树

##### 查找

- 在一颗二叉搜索树中查找一个具有给定关键字的结点

###### 代码实现

In [0]:
def tree_search(x, k):
  if x == None or k == x.key:
    return x
  return tree_search(x.left, k) if k < x.key else tree_search(x.right, k)

In [67]:
print("二叉搜索树为：\n {}".format(bst))

二叉搜索树为：
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2



In [68]:
tree_search(bst.root, 2)

2

###### 复杂度分析

- 执行过程
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200211190850.png width=700>
- $tree\_search$ 的运行时间为 $O(h)$，其中 $h$ 为树的高度

###### 迭代版本

In [69]:
%%writefile -a ch12/binary_search_tree.py


  def search(self, x, k):
    while x != None and k != x.key:
      x = x.left if k < x.key else x.right
    return x

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode

In [71]:
bst = BinarySearchTree(bst.root)
res = bst.search(bst.root, 2)
res

2

In [72]:
type(res)

ch12.binary_search_tree.BinaryTreeNode

- 对于大多数计算机来说，迭代版本的效率要更加高

#### 最大值和最小值

- 从根节点开始一直沿着左子节点向下遍历，直至遇到 $NIL$，即可找到最小元素
- 从根节点开始一直沿着右子节点向右遍历，直至遇到 $NIL$, 即可找到最大元素

###### 代码实现

In [73]:
%%writefile -a ch12/binary_search_tree.py


  def minimum(self, x):
    while x.left != None:
      x = x.left
    return x

Appending to ch12/binary_search_tree.py


In [74]:
%%writefile -a ch12/binary_search_tree.py


  def maximum(self, x):
    while x.right != None:
      x = x.right
    return x

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode

In [77]:
bst = BinarySearchTree(bst.root)
print("二叉搜索树为：\n {}".format(bst))

二叉搜索树为：
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2



In [78]:
bst.minimum(bst.root).key

2

In [79]:
bst.maximum(bst.root).key

20

#### 后继和前驱

##### 后继

- $x$ 的后继是指大于 $x.key$ 的最小的关键字的节点
- $x$ 如果有右子节点，则其后继为其右子树中的最小值
- $x$ 如果没有右子节点，则需要向上遍历，直至找到一个节点 $y$, 其中 $y$ 的左子节点也是 $x$ 的父辈节点或者是 $x$ 本身
（即 $x$ 在 $y$ 的左子树中）
  - 证明:
    1.  $y$ 必须为 $x$ 的父辈节点
       - 如果 $y$ 不是 $x$ 的父辈节点，设 $z$ 是 $x$ 和 $y$ 共同的父辈节点，由于 $y > x$，所以 $x$ 和 $y$ 分别位于 $z$ 的左子树和右子树，
       则 $x < z < y$，这与 $y$ 是 $x$ 的后继相矛盾，故 $y$ 必须为 $x$ 的父辈节点
    2.   $y.left$ 也必须是 $x$ 的父辈节点或者是 $x$ 本身
       - 如果 $y.left$ 不是 $x$ 的父辈节点也不是 $x$ 本身，则 $y.right$ 是 $x$ 的父辈节点，即 $x > y$， 与 $y$ 是 $x$ 的后继矛盾，故 $y.left$ 也必须是 $x$ 的父辈节点或 $x$ 本身
    3. 满足 1，2 后， $y$ 还必须是距离 $x$ 最近的节点
       - 如果 $y$ 不是距离 $x$ 最近的节点，设 $z$ 是满足 1,2， 且距离 $x$ 最近的节点，则 $z$ 位于 $y$ 的左子树中，即
       $x < z < y$，与 $y$ 是 $x$ 的后继相矛盾


###### 代码实现

In [80]:
%%writefile -a ch12/binary_search_tree.py


  def successor(self, x):
    if x.right != None:
      return self.minimum(x.right)
    y = x.parent
    while y != None and y.right == x:
      x = y
      y = x.parent
    return y

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode
bst = BinarySearchTree(bst.root)

In [82]:
print("二叉搜索树为：\n {}".format(bst))

二叉搜索树为：
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2



In [83]:
i = 15
print("the successor of {} is: {}".format(i, bst.successor(bst.search(bst.root, i)).key))

the successor of 15 is: 17


In [84]:
i = 13
print("the successor of {} is: {}".format(i, bst.successor(bst.search(bst.root, i)).key))

the successor of 13 is: 15


##### 前驱

- $x$ 的前驱是指小于 $x.key$ 的最大关键字的节点
- 如果 $x$ 存在左子节点，则其前驱是其左子树中的最大元素
- 如果 $x$ 不存在左子节点，则需要向上遍历，直至找到一个节点 $y$，其中 $y$ 的右子节点也是 $x$ 的父辈节点
(即 $x$ 在 $y$ 的右子树中)
  - 证明与后继的证明基本相同

###### 代码实现

In [85]:
%%writefile -a ch12/binary_search_tree.py


  def predecessor(self, x):
    if x.left != None:
      return self.maximum(x.left)
    y = x.parent
    while y != None and y.left == x:
      x = y
      y = x.parent
    return y

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode
bst = BinarySearchTree(bst.root)

In [87]:
print("二叉搜索树为：\n {}".format(bst))

二叉搜索树为：
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2



In [88]:
i = 15
print("the predecessor of {} is: {}".format(i, bst.predecessor(bst.search(bst.root, i)).key))

the predecessor of 15 is: 13


In [89]:
i = 13
print("the predecessor of {} is: {}".format(i, bst.predecessor(bst.search(bst.root, i)).key))

the predecessor of 13 is: 9


###### 定理 12.2
- 在高为 $h$ 的二叉搜索树上， `SEARCH, MAXIMUM, MINIMUM, SUCCESSSOR, PREDEDCESSOR` 可以在 $O(h)$ 的时间内完成

#### 代码运行图示

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200212092833.png width=700>

### 练习

#### 12.2-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213092957.png width=800>

- 由二叉树的性质可知，查找序列必须满足以下两个条件
  1. 如果 $a,b,c$ 是序列中连续出现的三项，则 $a\leq b \leq c$ 或 $a \geq b \geq c$
  2. 如果 $a$ 大于要查找的数值，那么之后序列中的所有值均小于等于 $a$，如果 $a$ 小于要查找的数值，则之后序列中的所有值均要大于等于 $a$
- 序列 c 中 $912 > 363,\  912 > 911$， 不符合条件 2
- 序列 e 中 $347 < 621 > 299$，不符合条件 1

#### 12.2-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213093955.png width=600>

In [0]:
def recursive_minimum(x):
  return recursive_minimum(x.left) if x.left != None else x

In [0]:
def recursive_maximum(x):
  return recursive_maximum(x.right) if x.right != None else x

In [92]:
print("二叉搜索树为：\n {}".format(bst))

二叉搜索树为：
 
|  |  20

|  18

|  |  17

15

|  |  |  13

|  |  |  |  9

|  |  7

|  6

|  |  |  4

|  |  3

|  |  |  2



In [93]:
recursive_minimum(bst.root).key

2

In [94]:
recursive_maximum(bst.root).key

20

#### 12.2-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213094604.png width=800>

- 假设二叉树如下
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213094745.png width=600>
- 如果要查找 4，可得 $B={1,3,4}, A={2}$， 则 $1 \in B, 2 \in A, 1 < 2$，不符合假设

#### 12.2-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213095240.png width=800>

- 证明：
  - 由于二叉搜索树有两个孩子，所以其后继为其右子树中的最小值，因此其后继不能有左孩子
  - 其前驱为其左子树是的最大值，所以不能有右孩子

#### 12.2-7
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213095547.png width=800>

- 证明：
  - 由于有 $n$ 个节点，所以运行时间至少为 $\Omega(n)$
  - 连续调用 `TREE-SUCCESSOR` 相当于遍历二叉树，则二叉树的每条边都会被遍历两次，由于二叉树的节点数比边数多 1，所以运行时间多为 $O(n)$
  - 综上可得，运行时间为 $\Theta(n)$

#### 12.2-8
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213100550.png width=800>

- 证明：
  1. 设 $k$ 次调用 `SUCCESSOR` 从 $x$ 遍历至 $y$， 由 12.2-7 可知，每条路径至多被遍历两次，则 $k$ 个结点，每个结点至多在路径上被经过 3 次，复杂度最多为 $3k$
  2. 设 $z$ 是 $x$ 和 $y$ 最底层的共用的父辈节点，则遍历的路径一定会经过 $z$。如果节点不在 $[x, y]$ 的范围内，则至多只会被遍历一次，而且这此结点只会出现 $x \rightarrow z$ 或 $y \rightarrow z$ 的路径上，复杂度至多为 $2h$
  3. 由 1，2 可知，总的时间复杂度为 $3k + 2h = O(k+h)$

#### 12.2-9
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213103718.png width=800>

- 证明：
  - 如果 $x$ 是 $y$ 的左子节点，则 $x$ 是 $y$ 的前驱，$y$ 是 $x$ 的后继，即 $y.key$ 是 $T$ 树是大于 $x.key$ 的最小关键字
  - 如果 $x$ 是 $y$ 的右子节点，则 $x$ 是 $y$ 的后继，$y$ 是 $x$ 的前驱，即 $y.key$ 是 $T$ 树是小于 $x.key$ 的最大关键字

## 12.3 插入和删除

- 插入和删除操作会引起由二叉搜索树表示的动态集合的变化，此时需要修改数据结构来反映这种变化，但修改需要保持二叉搜索树的性质成立

#### 插入

- 从根节点开始向下遍历，直至找到要插入的位置。在遍历的过程中，需要用一个指针 $x$ 记录一条向下的简单路径，同时保持遍历指针 $y$ 作为 $x$ 的父节点
- 示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200212095159.png width=700>

###### 代码实现

In [95]:
%%writefile -a ch12/binary_search_tree.py


  def insert(self, z):
    x, y = self.root, None
    while x != None:
      y = x
      x = x.left if z.key < x.key else x.right
    z.parent = y
    if y == None:
      self.root = z
    elif z.key < y.key:  # 不能用 y.left 是否为 None 来判断，因为 y 可能为叶子节点
      y.left = z
    else:
      y.right = z

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode

In [98]:
bst = BinarySearchTree()
for item in [12, 5, 18, 2, 9, 15, 19, 17]:
  bst.insert(BinaryTreeNode(key=item))
print("插入前二叉搜索树为：\n{}".format(bst))
i = 13
bst.insert(BinaryTreeNode(key=13))
print("插入{}后，二叉搜索树为：\n{}".format(i, bst))

插入前二叉搜索树为：

|  |  19

|  18

|  |  |  17

|  |  15

12

|  |  9

|  5

|  |  2

插入13后，二叉搜索树为：

|  |  19

|  18

|  |  |  17

|  |  15

|  |  |  13

12

|  |  9

|  5

|  |  2



#### 删除

##### 基本可分为四种情况:

###### 1. 如果 $z$ 没有左子节点，那么用其右子节点来替换 $z$，这个右子结点可以是 $NIL$

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200212105217.png width=400>

###### 2. 如果 $z$ 仅有一个子节点且为左子节点，那么用其左子节点来替换 $z$

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200212105501.png width=400>

###### 3. 
- $z$ 既有一个左子节点又有一个右子节点，需要查找 $z$ 的后继 $y$，$y$ 位于 $z$ 的右子树中并且没有左子节点
- 如果 $y$ 是 $z$ 的右子节点，可以直接用 $y$ 替换 $z$

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200212105957.png width=500>

###### 4.
- $z$ 既有一个左子节点又有一个右子节点，需要查找 $z$ 的后继 $y$，$y$ 位于 $z$ 的右子树中并且没有左子节点
- 如果 $y$ 不是 $z$ 的右子节点，此时需要用 $y$ 的右子节点替换 $y$，然后再用 $y$ 替换 $z$

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200212110221.png width=800>

##### 移动子树 TRANSPLANT

- TRANSPLANT 用一颗以 $v$ 为根的子树来替换一颗以 $u$ 为根的子树，结点 $u$ 的父节点变为节点 $v$ 的父节点， $v$ 变成 $u$ 的父节点对应的子节点

##### 代码实现

###### TRANSPLANT

In [99]:
%%writefile -a ch12/binary_search_tree.py



  def transplant(self, u, v):
    if u.parent == None:
      self.root = v
    elif u.parent.left == u:
      u.parent.left = v
    else:
      u.parent.right = v
    if v != None:
      v.parent = u.parent

Appending to ch12/binary_search_tree.py


- `TRANSPLANT` 没有处理 $u.left$ 和 $u.right$ 的更新，需要由调用者进行处理

###### DELETE

In [100]:
%%writefile -a ch12/binary_search_tree.py


  def delete(self, z):
    if z.left == None:
      self.transplant(z, z.right)
    elif z.right == None:
      self.transplant(z, z.left)
    else:
      y = self.minimum(z.right)
      if y.parent != z:
        self.transplant(y, y.right)
        y.right = z.right  # 将 y 移动至 z 的位置
        z.right.parent = y
      self.transplant(z, y)
      z.left.parent = y
      y.left = z.left

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode
bst = BinarySearchTree(bst.root)

In [102]:
bst = BinarySearchTree()
for item in [12, 5, 18, 2, 9, 15, 19, 17, 1]:
  bst.insert(BinaryTreeNode(key=item))
print("删除前二叉搜索树为：\n{}".format(bst))

i = 17
bst.delete(bst.search(bst.root, i))
print("删除 {} 后，二叉搜索树为：\n {}".format(i, bst))

i = 1
bst.delete(bst.search(bst.root, i))
print("删除 {} 后，二叉搜索树为：\n {}".format(i, bst))

i = 12
bst.delete(bst.search(bst.root, i))
print("删除 {} 后，二叉搜索树为：\n {}".format(i, bst))

删除前二叉搜索树为：

|  |  19

|  18

|  |  |  17

|  |  15

12

|  |  9

|  5

|  |  2

|  |  |  1

删除 17 后，二叉搜索树为：
 
|  |  19

|  18

|  |  15

12

|  |  9

|  5

|  |  2

|  |  |  1

删除 1 后，二叉搜索树为：
 
|  |  19

|  18

|  |  15

12

|  |  9

|  5

|  |  2

删除 12 后，二叉搜索树为：
 
|  |  19

|  18

15

|  |  9

|  5

|  |  2



##### 定理 12.3
- 在一颗高为 $h$ 的二叉搜索树上，实现 `INSERT` 和 `DELETE` 的运行时间为 $O(h)$

### 练习

#### 12.3-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213181316.png width=350>

In [0]:
def recursive_tree_insert(root, z, y=None):
  if root != None:
    if z.key < root.key:
      recursive_tree_insert(root.left, z, root)
    else:
      recursive_tree_insert(root.right, z, root)
  else:
    z.p = y
    if y == None:
      root = z
    elif z.key < y.key:
      y.left = z
    else:
      y.right = z


In [104]:
bst = BinarySearchTree()
for item in [12, 5, 18, 2, 9, 15, 19, 17]:
  bst.insert(BinaryTreeNode(key=item))
print("插入前二叉搜索树为：\n{}".format(bst))
i = 13
recursive_tree_insert(bst.root, BinaryTreeNode(key=i))
print("插入{}后，二叉搜索树为：\n{}".format(i, bst))

插入前二叉搜索树为：

|  |  19

|  18

|  |  |  17

|  |  15

12

|  |  9

|  5

|  |  2

插入13后，二叉搜索树为：

|  |  19

|  18

|  |  |  17

|  |  15

|  |  |  13

12

|  |  9

|  5

|  |  2



#### 12.3-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213193248.png width=700>

- 当二叉树的树深为 $n$ 时，会出现最坏情况，运行时间为 $\Theta(n^2)$
- 二叉树的高度 $h \geq \lfloor lgn \rfloor$，所以最好情况下，运行时间的上限为$O(nlgn)$，基于决策树模型可知，任何基于比较的排序算法的运行
时间下界为$\Omega(nlgn)$，所以，运行时间为 $\Theta(nlgn)$

#### 12.3-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213194026.png width=800>

- 不可交换，反例如下：
 - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213194450.png width=200>
 - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213194533.png width=200>

#### 12.3-5 中文版题目表述不清，换英文版的题目
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200213194743.png width=600>

###### search 算法

- search 算法只需要对左右节点进行操作，因此不需要修改

###### parent 算法

- 为了寻找 $x$ 的父节点，可以先找出以 $x$ 为根的二叉树的最大值 $y$
  - 如果 $y.succ$ 不存在，则说明 $y$ 是二叉树的最大值，说明 $x$ 在二叉树 $T$ 查找最大值的路径上
  - 如果 $y.succ$ 存在，由于 $y$ 一定没有右子节点，说明$y.succ$在$y$的父辈节点中，且$y.succ$所在的层级一定要高于 $x$，而且
  $y.succ.left$ 一定为 $x$ 的父节点，或者为 $x$ 本身
    - 如果 $y.succ.left$ 为 $x$ 本身，则 $y.succ$ 即为 $x$ 的父节点
    - 否则， $x$ 位于以 $y.succ.left$ 为根的二叉树的查找最大值的路径上  

###### insert 算法

- insert 算法基本不用改变，但需要更新相应节点的后继值
- 设插入的值为 $z$，最终遍历到的叶子节点为 $y$
  - 如果 $z$ 成为 $y$ 的右子节点，说明 $y$ 没有右子节点；则$y.succ$ 为 $y$ 的父辈节点，则 $y.succ = z$, 
  而 $z.succ = y.succ$
  - 如果 $z$ 成为 $y$ 的左子节点，说明 $y$ 没有左子节点，则其前驱$x$在其父辈节点中，且 $y$ 在 $x$ 的左子树中；
  则 $z.succ$ 为 y，而 $x.succ$ 的后继为 $y$

###### predecessor 算法

- 查找前驱，直接调用 $parent$ 算法，步骤与原算法基本相同

###### delete 算法

- 设要删除的节点为 $x$
- 设 $x$ 的前驱为 $y$，则需要将 $y.succ$ 设为 $x.succ$，其它操作不需要改变

###### 代码实现

In [105]:
%%writefile ch12/binary_search_tree_successor.py
"""结点没有父指向父结点的指针，只有指向后继的指针"""
from ch12.binary_search_tree import BinaryTreeNode, BinarySearchTree


class BinaryTreeNodeSuccessor(BinaryTreeNode):
  def __init__(self, left=None, right = None, succ=None, key=None):
    self.left = left
    self.right = right
    self.succ = succ
    self.key = key


class BinarySearchTreeSuccessor(BinarySearchTree):
  def __init__(self, root=None):
    super().__init__(root)

  def search(self, x, k):
    """搜索只需对左右节点进行操作，直接继承即可"""
    return super().search(x, k)

  def parent(self, x):
    if x == self.root:
      return None
    y = self.maximum(x).succ
    if y == None:
      y = self.root
    else:
      if y.left == x:
        return y
      y = y.left
    while y.right != x:
      y = y.right
    return y

  def insert(self, z):
    x = self.root
    y = None
    predecessor = None
    while x != None:
      y = x
      if z.key < x.key:
        x = x.left
      else:
        predecessor = x  # 最后一次转向右子节点的点，一定为最终 y 的前驱
        x = x.right
    if y == None:
      self.root = z
    else:
      if z.key < y.key:
        y.left = z
        z.succ = y
        if predecessor != None:
          predecessor.succ = z 
      else:
        y.right = z
        z.succ = y.succ
        y.succ = z
  
  def predecessor(self, x):
    if x.left != None:
      return self.maximum(x.left)
    y = self.parent(x)
    while y != None and y.left == x:
      x = y
      y = self.parent(y)
    return y

  def transplant(self, u, v):
    u_parent = self.parent(u)
    if u_parent == None:
      self.root = v
    elif u == u_parent.left:
      u_parent.left = v
    else:
      u_parent.right = v
      
  def delete(self, z):
    self.predecessor(z).succ = z.succ
    if z.left == None:
      self.transplant(z, z.right)
    elif z.right == None:
      self.transplant(z, z.left)
    else:
      if z.right != z.succ:
        self.transplant(z.succ, z.succ.right)
        z.succ.right = z.right
      self.transplant(z, z.succ)
      z.succ.left = z.left

Overwriting ch12/binary_search_tree_successor.py


In [0]:
from ch12.binary_search_tree_successor import BinarySearchTreeSuccessor, BinaryTreeNodeSuccessor

In [107]:
bst = BinarySearchTreeSuccessor()
for item in [12, 5, 18, 2, 9, 15, 19, 17]:
  bst.insert(BinaryTreeNodeSuccessor(key=item))
print("插入前二叉搜索树为：\n{}".format(bst))

i = 15
bst.delete(bst.search(bst.root, i))
print("册除{}后，二叉搜索树为：\n{}".format(i, bst))

i = 13
bst.insert(BinaryTreeNodeSuccessor(key=i))
print("插入{}后，二叉搜索树为：\n{}".format(i, bst))

i = 12
bst.delete(bst.search(bst.root, i))
print("册除{}后，二叉搜索树为：\n{}".format(i, bst))

插入前二叉搜索树为：

|  |  19

|  18

|  |  |  17

|  |  15

12

|  |  9

|  5

|  |  2

册除15后，二叉搜索树为：

|  |  19

|  18

|  |  17

12

|  |  9

|  5

|  |  2

插入13后，二叉搜索树为：

|  |  19

|  18

|  |  17

|  |  |  13

12

|  |  9

|  5

|  |  2

册除12后，二叉搜索树为：

|  |  19

|  18

|  |  17

13

|  |  9

|  5

|  |  2



#### 12.3-6
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200214132618.png width=800>

 ###### 采用 $y$ 作为前驱

In [108]:
%%writefile -a ch12/binary_search_tree.py



  def delete_predecessor(self, z):
    """删除时如果左，右子树均存在，则删除前驱而不是后继"""
    if z.left == None:
      self.transplant(z, z.right)
    elif z.right == None:
      self.transplant(z, z.left)
    else:
      y = self.maximum(z.left)
      if y != z.left:
        self.transplant(y, y.left)
        y.left = z.left
        z.left.parent = y
      self.transplant(z, y)
      y.right = z.right
      y.right.parent = y

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode
bst = BinarySearchTree(bst.root)

In [110]:
bst = BinarySearchTree()
for item in [12, 5, 18, 2, 9, 15, 19, 17, 1]:
  bst.insert(BinaryTreeNode(key=item))
print("删除前二叉搜索树为：\n{}".format(bst))


i = 5
bst.delete_predecessor(bst.search(bst.root, i))
print("y 为前驱,删除 {} 后，二叉搜索树为：\n {}".format(i, bst))

i = 12
bst.delete_predecessor(bst.search(bst.root, i))
print("y 为前驱,删除 {} 后，二叉搜索树为：\n {}".format(i, bst))

删除前二叉搜索树为：

|  |  19

|  18

|  |  |  17

|  |  15

12

|  |  9

|  5

|  |  2

|  |  |  1

y 为前驱,删除 5 后，二叉搜索树为：
 
|  |  19

|  18

|  |  |  17

|  |  15

12

|  |  9

|  2

|  |  1

y 为前驱,删除 12 后，二叉搜索树为：
 
|  |  19

|  18

|  |  |  17

|  |  15

9

|  2

|  |  1



###### 公平的策略

- 使用随机数生成来确定要使用前驱还是后继

In [111]:
%%writefile -a ch12/binary_search_tree.py


  def delete_random(self, z):
    if random.randint(0,1) == 0:
      self.delete(z)
    else:
      self.delete_predecessor(z)

Appending to ch12/binary_search_tree.py


In [0]:
# 重新加载相应的模块
imp.reload(ch12.binary_search_tree) 
from ch12.binary_search_tree import BinarySearchTree, BinaryTreeNode
bst = BinarySearchTree(bst.root)

In [114]:
bst = BinarySearchTree()
for item in [12, 5, 18, 2, 9, 15, 19, 17, 1]:
  bst.insert(BinaryTreeNode(key=item))
print("删除前二叉搜索树为：\n{}".format(bst))


i = 5
bst.delete_random(bst.search(bst.root, i))
print("随机选择,删除 {} 后，二叉搜索树为：\n {}".format(i, bst))

i = 12
bst.delete_random(bst.search(bst.root, i))
print("随机选择,删除 {} 后，二叉搜索树为：\n {}".format(i, bst))

删除前二叉搜索树为：

|  |  19

|  18

|  |  |  17

|  |  15

12

|  |  9

|  5

|  |  2

|  |  |  1

随机选择,删除 5 后，二叉搜索树为：
 
|  |  19

|  18

|  |  |  17

|  |  15

12

|  |  9

|  2

|  |  1

随机选择,删除 12 后，二叉搜索树为：
 
|  |  19

|  18

|  |  |  17

|  |  15

9

|  2

|  |  1



## 12.4 随机构建的二叉搜索树

- 对于有 $n$ 个结点的二叉搜索树，其高度 $h \ge \lfloor lgn\rfloor$
- 对于随机构建的二叉搜索树，其平均性能更接近于最好情形

#### 定理12.4 
- 一颗有 $n$ 个不同关键字的随机构建二叉树的期望高度为 $O(lgn)$

##### 证明

  - 定义 $X_n$ 表示一棵有 $n$ 个不同关键字的随机构建二叉搜索树的高度
  - 定义**指数高度** $Y_n = 2^{X_n}$
    - 采用指数高度的目的是将递归式由加法的形式变为乘法，更易推导出上限
  - 定义 $R_n$ 表示树根的关键字在 $n$ 个不同的关键字集合中的秩，即关键字排好序后，这个关键字所占据的位置
  - 如果 $R_n = i$， 则左子树有 $i-1$ 个结点，右子树有 $n-i$ 个节点，则有: $$Y_{n}=2 \cdot \max \left(Y_{i-1}, Y_{n-i}\right)$$
  - 定义指示器随机变量 $Z_{n,i} = I\{R_n=i\}$，可得 $$E[Z_{n,i}]=1/n$$
  - $$Y_{n}=\sum_{i=1}^{n} Z_{n, i}\left(2 \cdot \max \left(Y_{i-1}, Y_{n-i}\right)\right)$$
    - 只要证明 $E[Y_n]$ 是 $n$ 的一个多项式，即可推出 $E[X_n] = O(lgn)$
  - $$
    \begin{aligned}
    \mathrm{E}\left[Y_{n}\right] &=\mathrm{E}\left[\sum_{i=1}^{n} Z_{n, i}\left(2 \cdot \max \left(Y_{i-1}, Y_{n-i}\right)\right)\right] \\
    &=\sum_{i=1}^{n} \mathrm{E}\left[Z_{n, i}\left(2 \cdot \max \left(Y_{i-1}, Y_{n-i}\right)\right)\right] &&(期望的线性性质)\\
    &=\sum_{i=1}^{n} \mathrm{E}\left[Z_{n, i}\right] \mathrm{E}\left[2 \cdot \max \left(Y_{i-1}, Y_{n-i}\right)\right] &&(独立性)\\
    &=\sum_{i=1}^{n} \frac{1}{n} \cdot \mathrm{E}\left[2 \cdot \max \left(Y_{i-1}, Y_{n-i}\right)\right] \\
    &=\frac{2}{n} \sum_{i=1}^{n} \mathrm{E}\left[\max \left(Y_{i-1}, Y_{n-i}\right)\right] \\
    & \leq \frac{2}{n} \sum_{i=1}^{n}\left(\mathrm{E}\left[Y_{i-1}\right]+\mathrm{E}\left[Y_{n-i}\right]\right) \\
    & \le \frac{4}{n} \sum_{i=0}^{n-1}\mathrm{E}\left[Y_{i}\right]
    \end{aligned} 
    $$
  - 通过代入法，可证明 $\mathrm{E}[Y_n] \le cn^3$
    - $n = 1$ 时, 当 $c \leq 1$ 时，不等式显然成立
    - 假设对于所有小于 $n-1$ 的数，不等式均成立，即可得
    $$\begin{aligned}
      E\left[Y_{n}\right] &=\frac{4}{n} \sum_{k=0}^{n-1} E\left[Y_{k}\right] \\
      & \leq \frac{4}{n} \sum_{k=0}^{n-1} c k^{3} \\
      & \leq \frac{4 c}{n} \int_{0}^{n} x^{3} d x \\
      &=\frac{4 c}{n}\left(\frac{n^{4}}{4}\right) \\
      &=c n^{3}
      \end{aligned}
    $$
    - 由上式可知，不等式对于 $n$ 也成立，证毕
  - 由于 $f(x) = 2^x$ 为凸函数，应用 Jensen 不等式，可得 
  $$   \begin{aligned}
      2^{\mathrm{E}\left[X_{n}\right]} & \leq \mathrm{E}\left[2^{X_{n}}\right] \\
      &=\mathrm{E}\left[Y_{n}\right]
      \end{aligned}
  $$ 
  - 综上可得：$$\mathrm{E}[X_n] = O(lgn)$$

##### Jensen 不等式的证明

###### 凸函数
- $f: \mathbb{R} \rightarrow \mathbb{R}$，如果 $\alpha, \beta \ge 0$ 且 $\alpha + \beta = 1$， $\forall x, y \in \mathbb{R}$，有下式成立，则说明 $f$ 为凸函数
  - $$f(\alpha x+\beta y) \leq \alpha f(x)+\beta f(y)$$

- 如果一元函数 $f$ 的二阶导数存在，且恒大于0，则 $f$ 为凸函数

###### 引理 1
- 如果 $f: \mathbb{R} \rightarrow \mathbb{R}$ 为凸函数， $\alpha_{1}, \alpha_{2}, \dots, \alpha_{n}$ 为非负实数且满足 $\sum_{k} \alpha_{k}=1$，则 $\forall x_{1}, x_{2}, \ldots, x_{n} \in \mathbb{R}$，有：
  - $$f\left(\sum_{k=1}^{n} \alpha_{k} x_{k}\right) \leq \sum_{k=1}^{n} \alpha_{k} f\left(x_{k}\right)$$

- 证明：可通过数学归纳法和凸函数的相关定义证得
  - $n=1$ 时显然成立
  - 归纳步骤
    - $$
\begin{aligned}
f\left(\sum_{k=1}^{n} \alpha_{k} x_{k}\right) &=f\left(\alpha_{n} x_{n}+\left(1-\alpha_{n}\right) \sum_{k=1}^{n-1} \frac{\alpha_{k}}{1-\alpha_{n}} x_{k}\right) &&(代数变换)\\
& \leq \alpha_{n} f\left(x_{n}\right)+\left(1-\alpha_{n}\right) f\left(\sum_{k=1}^{n-1} \frac{\alpha_{k}}{1-\alpha_{n}} x_{k}\right) &&(凸函数的性质)\\
& \leq \alpha_{n} f\left(x_{n}\right)+\left(1-\alpha_{n}\right) \sum_{k=1}^{n-1} \frac{\alpha_{k}}{1-\alpha_{n}} f\left(x_{k}\right) &&(归纳的假设)\\
&=\sum_{k=1}^{n} \alpha_{k} f\left(x_{k}\right)
\end{aligned}
$$

###### 引理 2
- 当 $n \rightarrow \infty$ 时，引理 1 也成立，即 $$f\left(\sum_{k=1}^{\infty} \alpha_{k} x_{k}\right) \leq \sum_{k=1}^{\infty} \alpha_{k} f\left(x_{k}\right)$$

- 证明
  - 由引理 1 可得，$\forall n \ge 1$，有：
    - $$f\left(\sum_{k=1}^{n} \frac{\alpha_{k}}{\sum_{i=1}^{n} \alpha_{i}} x_{k}\right) \leq \sum_{k=1}^{n} \frac{\alpha_{k}}{\sum_{i=1}^{n} \alpha_{i}} f\left(x_{k}\right)$$
  - 上式两边同时取极限，可得：
    - $$\lim _{n \rightarrow \infty} f(\underbrace{\frac{1}{\sum_{i=1}^{n} \alpha_{i}}}_{\rightarrow 1} \underbrace{\sum_{k=1}^{n} \alpha_{k} x_{k}}_{\rightarrow \sum_{k=1}^{\infty} \alpha_{k} x_{k}})
    \le \lim _{n \rightarrow \infty} \underbrace{\frac{1}{\sum_{i=1}^{n} \alpha_{i}}}_{\rightarrow 1} \underbrace{\sum_{k=1}^{n} \alpha_{k} f\left(x_{k}\right)}_{\rightarrow \sum_{k=1}^{\infty} \alpha_{k} f\left(x_{k}\right)}$$

###### Jensen 不等式
- $f$ 为凸函数， $X$ 为随机变量，则 $f(E[X]) \le E(f(X))$

- 证明
  - $$\begin{aligned}
f(E[X]) &=f\left(\sum_{k=-\infty}^{\infty} k \cdot \operatorname{Pr}\{X=k\}\right)  &&(期望的定义)\\
& \leq \sum_{k=-\infty}^{\infty} f(k) \cdot \operatorname{Pr}\{X=k\} &&(引理2) \\
& = \sum_{y \in f(X)} {y \sum_{x: f(x) = y}Pr\{X=x\}} &&(相当于对上式进行分组) \\
& = E(f(X))
\end{aligned}$$

### 12.2-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200214143507.png width=350>

- $f^{''}(x) = 2^{x}ln^2(2) \geq 0$，所以可得 $f(x)$ 是凸的

## 思考题

### 12-1 带有相同关键字的二叉搜索树
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200214145647.png width=800>

###### a

最终树高将为 $n$，时间复杂度为 $\sum_{i=1}^{n}i = \Theta(n^2)$

###### b

- 此时树将平衡，树高为 $lg(n)$, 则时间复杂度为 $\sum_{i=1}^{n}(lgi)$
  - 设 $y = \sum_{i=1}^{n}(lgi)$，可得：
    - $$2^y = \prod_{i=1}^{n}2^(lg\ i) = n!  \rightarrow y = lg(n!) = \Theta(nlgn)$$

###### c

- 此时树高恒为 0， 只需将各节点插入到列表中即可，时间复杂度为 $\Theta(n)$

###### d

- 最坏的情况下，每次的随机值均相同，此时树高为 $n$，运行时间为 $\Theta(n^2)$
- 平均情况下， $x.left$ 与 $x.right$ 出现的概率相同，即二叉树将是平衡的，树高为 $lg(n)$，则平均的运行时间为 $\Theta(nlgn)$

### 12-2 基数树
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200214154323.png width=700>

- 首先需要构造基数树，设 $a$ 有 $p$ 位，则在将 $a$ 插入基数树的过程中，需要经过 $p$ 次比较，如果 $S$ 中各个串的长度和为 $n$，则在构造基树是，要进行的比较次数为 $n$
- 基树构造完成后，只需执行一次先序遍历，即可按次序输出排序后的序列。设基数树是有$x$ 个结点，则此步需用时 $\Theta(x)$
- 由于 $x\leq n$，所以可以在 $\Theta(n)$ 的时间内按字典序对 $S$ 进行排序

### 12-3 随机构建二叉搜索树中的平均结点深度
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200214203616.png width=700>

###### a

根据定义即可证明

###### b

$$
\begin{aligned}
P(T)&=\sum_{x \in T} d(x, T)=\sum_{x \in T_{L}} d(x, T)+\sum_{x \in T_{R}} d(x, T) && (根节点的高度为0)\\
&=\sum_{x \in T_{L}}\left(d\left(x, T_{L}\right)+1\right)+\sum_{x \in T_{R}}\left(d\left(x, T_{R}\right)+1\right)\\
&=\sum_{x \in T_{L}} d\left(x, T_{L}\right)+\sum_{x \in T_{R}} d\left(x, T_{R}\right)+n-1\\
&=P\left(T_{L}\right)+P\left(T_{R}\right)+n-1
\end{aligned}
$$

###### c

- 证明：
  - 由定义可知：$P(n) = \mathrm{E}(P(T))$
  - 定义指示器随机变量 $Z_{n,i} = I\{$ 左子树有 $i$ 个节点，右子树有 $n-i-1$ 个节点$\}$
    - $$E(Z_{n,i}) = \frac{1}{n}$$
    - $$\begin{aligned}
    P(n) &= \mathrm{E}(P(T)) = E(\sum_{i=0}^{n-1}{Z_{n,i}(P(i) + P(n-i-1) + n - 1)}) && (b 中的结论)\\
       &= \frac{1}{n} \sum_{i=0}^{n-1} (P(i)+P(n-i-1)+n-1)
    \end{aligned}$$

###### d

$$\begin{aligned}
P(n) &= \frac{1}{n} \sum_{i=0}^{n-1} P(i)+P(n-i-1)+n-1\\
   &= \frac{1}{n}\left(\sum_{i=0}^{n-1} P(i)+\sum_{i=0}^{n-1} P(n-i-1)+\sum_{i=0}^{n-1}(n-1)\right) 
   &&(将第二项中的 n-i-1 替换为j) \\
   &= \frac{1}{n}\left(\sum_{i=0}^{n-1} P(i)+\sum_{j=0}^{n-1} P(j)+n(n-1)\right) \\
   &= \frac{2}{n} \sum_{i=0}^{n-1} (P(i)+n-1)
\end{aligned}$$

###### e

- 证明：
  - $P(n)$ 的形式基本与思考题 7-3 中的形式相同，按照相同的方法，可证得 $$P(n)=O(nlgn)$$

###### f

1. 先选根元素做为主元，然后选择左子节点作为左边分区的主元，右子节点作为右边分区的主元（如果左子节点，或者右子节点不存在，说明左边分区和右
边分区也为空）。
2. 对左、右子树分别递归的执行 1，即可达到题目要求

### 12-4 不同二叉树的数目
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200214214136.png width=700>

###### a

- 证明：
  - 根节点拥有左、右两棵子树，所以可以推得： $$b_{n}=\sum_{k=0}^{n-1} b_{k} b_{n-1-k}$$

###### b

- 证明：
  - $$
\begin{aligned}
B(x)^{2} &=\left(b_{0} x^{0}+b_{1} x^{1}+b_{2} x^{2}+\cdots\right)^{2} \\
&=b_{0}^{2} x^{0}+\left(b_{0} b_{1}+b_{1} b_{0}\right) x^{1}+\left(b_{0} b_{2}+b_{1} b_{1}+b_{2} b_{0}\right) x^{2}+\cdots \\
&=\sum_{k=0}^{0} b_{k} b_{0-k} x^{0}+\sum_{k=0}^{1} b_{k} b_{1-k} x^{1}+\sum_{k=0}^{2} b_{k} b_{2-k} x^{2}+\cdots
\end{aligned}
$$
  - $$
  \begin{aligned}
x B(x)^{2}+1 &=1+\sum_{k=0}^{0} b_{k} b_{1-1-k} x^{1}+\sum_{k=0}^{2} b_{k} b_{2-1-k} x^{3}+\sum_{k=0}^{2} b_{k} b_{3-1-k} x^{2}+\cdots \\
&=1+b_{1} x^{1}+b_{2} x^{2}+b_{3} x^{3}+\cdots &&(根据 a 中的结论)\\
&=b_{0} x^{0}+b_{1} x^{1}+b_{2} x^{2}+b_{3} x^{3}+\cdots \\
&=\sum_{n=0}^{\infty} b_{n} x^{n} \\
&=B(x)
\end{aligned}
  $$



###### c

- 证明：
  - 设 $f(x) = \sqrt{1-4 x}$
    - $$
  \begin{aligned}
  f^{(k)}(0) &=  2 \cdot(1 \cdot 2) \cdot(3 \cdot 2) \cdot(5 \cdot 2) \cdots \\
  &=2^{k} \cdot \prod_{i=0}^{k-2}(2 k+1) \\
  &=2^{k} \cdot \frac{(2(k-1)) !}{2^{k-1}(k-1) !} && (将偶数全部除掉，即为奇数连乘) \\
  &=\frac{2(2(k-1)) !}{(k-1) !}
  \end{aligned}
  $$
  - 由泰勒不等式可得：
    - $$\begin{aligned}
       f(x)=\sum_{k=0}^{\infty} \frac{f^{(k)}(0)}{k !}x^{k} = 1 + \sum_{k=1}^{\infty}{\frac{2(2(k-1)) !}{k !(k-1) !}x^k}
       \end{aligned}$$
  - 由 b 中的结论可得：
    - $$
    \begin{aligned}
      B(x) &=\frac{1}{2 x}(1-f(x)) \\
      &= \sum_{n=1}^{\infty} \frac{(2(n-1)) !}{(n-1) ! n !} x^{n-1} &&(令 n = n - 1) \\
      &=\sum_{n=0}^{\infty} \frac{(2 n) !}{(n+1) ! n !} x^n \\
      &=\sum_{n=0}^{\infty} \frac{1}{n+1} \frac{(2 n) !}{n ! n !} x^n \\
      &=\sum_{n=0}^{\infty} \frac{1}{n+1}\left(\begin{array}{c}
      {2 n} \\
      {n}
      \end{array}\right) x^n
      \end{aligned}
    $$

###### d

- 证明：
  - 利用斯特林不等式
    - $$n !=\sqrt{2 \pi n}\left(\frac{n}{\mathrm{e}}\right)^{n}\left(1+\Theta\left(\frac{1}{n}\right)\right)$$
  - $$
  \begin{aligned}
b_{n} &=\frac{1}{n+1} \frac{(2 n) !}{n ! n !} \\
& \approx \frac{1}{n+1} \frac{\sqrt{4 \pi n}(2 n / e)^{2 n}}{2 \pi n(n / e)^{2 n}} \\
&=\frac{1}{n+1} \frac{4^{n}}{\sqrt{\pi n}} \\
&=\left(\frac{1}{n}+\left(\frac{1}{n+1}-\frac{1}{n}\right)\right) \frac{4^{n}}{\sqrt{\pi n}} \\
&=\left(\frac{1}{n}-\frac{1}{n^{2}+n}\right) \frac{4^{n}}{\sqrt{\pi n}} \\
&=\frac{1}{n}\left(1-\frac{1}{n+1}\right) \frac{4^{n}}{\sqrt{\pi n}} \\
&=\frac{4^{n}}{\sqrt{\pi} n^{3 / 2}}(1+O(1 / n))
\end{aligned}
  $$